# 4.2.4.3 PyTorch (MNIST)

In [0]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from tqdm import tqdm_notebook as tqdm

In [0]:
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.a1 = nn.ReLU()
        self.d1 = nn.Dropout(0.5)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.a2 = nn.ReLU()
        self.d2 = nn.Dropout(0.5)
        self.l3 = nn.Linear(hidden_dim, hidden_dim)
        self.a3 = nn.ReLU()
        self.d3 = nn.Dropout(0.5)
        self.l4 = nn.Linear(hidden_dim, output_dim)

        self.layers = [self.l1, self.a1, self.d1,
                       self.l2, self.a2, self.d2,
                       self.l3, self.a3, self.d3,
                       self.l4]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        return x

In [0]:
np.random.seed(123)
torch.manual_seed(123)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# 1. データの準備
root = os.path.join('~', '.torch', 'mnist')
transform = transforms.Compose([transforms.ToTensor(),
                                lambda x: x.view(-1)])
mnist_train = datasets.MNIST(root=root,
                             download=True,
                             train=True,
                             transform=transform)
mnist_test = datasets.MNIST(root=root,
                            download=True,
                            train=False,
                            transform=transform)

train_dataloader = DataLoader(mnist_train,
                              batch_size=100,
                              shuffle=True)
test_dataloader = DataLoader(mnist_test,
                             batch_size=100,
                             shuffle=False)


Extracting /root/.torch/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.torch/mnist/MNIST/raw



Extracting /root/.torch/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.torch/mnist/MNIST/raw



Extracting /root/.torch/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.torch/mnist/MNIST/raw



Extracting /root/.torch/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.torch/mnist/MNIST/raw
Processing...
Done!


In [0]:
# 2. モデルの構築
model = DNN(784, 200, 10).to(device)

In [10]:
# 3. モデルの学習
criterion = nn.CrossEntropyLoss()
optimizer = optimizers.SGD(model.parameters(), lr=0.01)

def compute_loss(t, y):
    return criterion(y, t)

def train_step(x, t):
    model.train()
    preds = model(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad() # 勾配の初期化
    loss.backward() # 勾配計算
    optimizer.step() # パラメータの更新

    return loss, preds

epochs = 30

for epoch in tqdm(range(epochs)):
    train_loss = 0.
    train_acc = 0.

    for (x, t) in train_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        train_loss += loss.item()
        train_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)

    print('epoch: {}, loss: {:.3}, acc: {:.3f}'.format(
        epoch+1,
        train_loss,
        train_acc
    ))

ストリーミング出力は最後の 5000 行に切り捨てられました。
        [ 4.7217e-01, -3.1575e+00, -2.5973e+00, -1.3396e-01,  2.6701e-01,
          3.2836e+00, -2.0163e+00,  2.8570e-01,  1.4033e+00,  2.0320e+00],
        [ 1.0252e+01, -1.2217e+01,  2.8653e+00,  4.1563e+00, -6.3851e+00,
          5.9244e+00,  5.1711e-01, -9.4080e-01,  1.5633e+00, -2.1064e+00],
        [ 2.1221e+00, -3.3432e+00, -8.2432e-01,  6.2798e-01, -6.1555e-01,
          2.6561e+00, -3.0606e-01, -5.3635e-01,  7.1147e-01,  1.0338e-01],
        [ 3.6553e+00, -9.6718e+00, -2.0639e-01,  3.2430e+00, -4.8157e+00,
          7.8863e+00, -2.0179e+00, -4.2344e+00,  6.8446e+00, -4.3486e-01],
        [-2.3611e+00,  3.0118e+00,  2.6869e+00,  1.9325e+00, -2.2536e+00,
         -1.7010e+00, -1.2313e+00,  6.2385e-01,  1.7334e-01, -1.1600e+00],
        [-1.5619e-01, -1.4063e+00,  9.2830e+00,  2.8969e+00, -4.7868e+00,
         -1.7831e+00,  6.6123e-01, -2.0032e+00,  2.0922e+00, -3.7926e+00],
        [ 1.3015e+00, -1.8757e+00,  6.4060e-01,  5.8197e-01, -6.1742e-01,


KeyboardInterrupt: ignored

In [0]:
# 4. モデルの評価
def test_step(x, t):
    model.eval()
    preds = model(x)
    loss = criterion(preds, t)

    return loss, preds

test_loss = 0.
test_acc = 0.

for (x, t) in test_dataloader:
    x, t = x.to(device), t.to(device)
    loss, preds = test_step(x, t)
    test_loss += loss.item()
    test_acc += \
        accuracy_score(t.tolist(),
                       preds.argmax(dim=-1).tolist())

test_loss /= len(test_dataloader)
test_acc /= len(test_dataloader)
print('test_loss: {:.3f}, test_acc: {:.3f}'.format(
    test_loss,
    test_acc
))

test_loss: 0.127, test_acc: 0.962
